# Syrtis Demonstration - Subterranean Heat Flows
# Heat Losses from Buried Habitats and Habitats inside Caves

Many modern proposals for near-future Mars and lunar habitation rely on habitats and crewed volumes located underground or beneath thick layers of regolith for radiation protection. This has substantial benefits for reducing launched mass while reducing crew health effects, as shielding of a few metres of locally-acquired regolith can reduce radiation levels in the habitat to safe levels without requiring many tonnes of water or metal to be launched as part of the habitat assembly. However the low thermal conductivity of regoliths, and the large thicknesses required, make this a challenging thermal design scenario. Syrtis allows for the analysis of habitats under layers of regolith, buried entirely below the surface and inside large lava tubes.

In [1]:
from syrtis import *
from syrtis.configuration import Configuration
from syrtis.solver import Solver
from syrtis.material import Solid, ConstrainedIdealGas
from syrtis.habitat import Habitat
import numpy as np
import matplotlib.pyplot as plt

### Setup and Configuration
The syrtis modules are imported. The two solid materials used for the habitat (aluminium and a generic plastic). Two gases are needed: the internal habitat air at 101kPa, and the higher pressure air which fills inflatable spaces inside the habitat walls.

The values of regolith thermal conductivity, and thermal conductivity of the sintered regolith, are hard to come by and appear somewhat variable. There appears to be no existing high-quality data on the thermal properties of sintered Martian regolith whatsoever, so values for [laser-sintered lunar regolith simulant](https://core.ac.uk/download/pdf/227532695.pdf) are used instead. Similarly, measured values of unmodified soil conductivity vary from just [0.039W/m/K in the top 30cm of soil at the InSight landing site](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021JE006861) to [0.085W/m/K from the northerly Phoenix site](https://www.lpi.usra.edu/meetings/lpsc2009/pdf/1125.pdf). This compares to laboratory values of [2.94W/m/K for the solid particle](https://art.torvergata.it/retrieve/handle/2108/51616/132578/04%20PSS.pdf). Considering this, the values below have been selected.

For the sintered regolith: $\lambda = 1.0 W/m\cdot K$, $rho = 2300 kg/m^3$

For the unsintered regolith near the surface: $\lambda = 0.15 W/m\cdot K$, $\rho = 1100 kg/m^3$

For the virgin rock around the lava tube: $\lambda = 2.0 W/m\cdot K$, $\rho = 3000 kg/m^3$.

Data on thermal and wind conditions in lava tubes is still an open question, but [simulation suggests](https://www.sciencedirect.com/science/article/pii/S0019103510001491?via=ihub) that temperatures of both air and ground would be stable near the coldest air temperature on the surface, and that wind inside would close to zero.

In [2]:
# Materials definitions
plastic = Solid("Generic plastic", 10, 1300, 1420, 0.5)
open_cell_foam = Solid("Generic open-cell polyurethane foam", 0.036, 10, 1500, 0.5)

internal_air = ConstrainedIdealGas("STP Air", 101325, 29, 0.71, 17.9e-6, 1010, 0.0252)
inflation_air = ConstrainedIdealGas("Air at 2bar", 202630, 29, 0.71, 17.9e-6, 1010, 0.0252)

sintered_regolith = Solid("Laser sintered lunar regolith", 1.0, 2300, 2000, 0.29)

mount_sharp_summer = Configuration("Curiosity landing area, Mount Sharp, at noon in summer",
 "constant temperature", 290, 0.15, 0.29, 250, 610, 5, "axial", 70, 90, 610, T_habitat=293)
mount_sharp_winter = Configuration("Curiosity landing area, Mount Sharp, at night in winter",
 "constant temperature", 165, 0.15, 0.29, 110, 610, 5, "axial", 0, 90, 0, T_habitat=293)

mount_sharp_lava_tube_summer = Configuration("Lava tube in summer",
 "constant temperature", 200, 2.0, 0.29, 200, 610, 0.02, "axial", 0, 0, 0, T_habitat=293)

## Habitat Geometry

Construction below the surface in many cases neccessitates inflatable habitats, as large solid modules cannot be easily transported and lifted on the surface. In addition, the relative lightweight nature of inflatable habitats makes them a better candidate for moving into trenches and caves. Little reliable data on the wall assemblies of inflatable habitats exist, so the following wall stackup has been used, based on TransHab designs

0.5mm of plastic (ETFE, a pressure bladder)

2mm of air at 2 bar, as a structural reinforcement section

0.5mm of plastic, a second pressure bladder

2mm of foam (approximating the Kevlar felt layer used to separate bladder layers on Transhab)

2mm of plastic (Kevlar, the structural layer)

10mm of foam (a low-density material used to buffer the pressure vessel from the rough surface of the regolith)

In [3]:
HAB_baseline = Habitat(orientation="horizontal", length=8, endcap_type="hemisphere")

HAB_baseline.create_static_shell(internal_air, 1.50)

HAB_baseline.create_static_shell(plastic, 0.5e-3)
HAB_baseline.create_static_shell(inflation_air, 2e-3)
HAB_baseline.create_static_shell(plastic, 0.5e-3)

HAB_baseline.create_static_shell(open_cell_foam, 2e-3)
HAB_baseline.create_static_shell(plastic, 2e-3)
HAB_baseline.create_static_shell(open_cell_foam, 10e-3)

HAB_sintered_shield = copy.deepcopy(HAB_baseline)
HAB_sintered_shield.create_static_shell(sintered_regolith, 1.5)
HAB_sintered_shield.create_ground_level(habitat_axis_height=1.4)

HAB_tunnel = copy.deepcopy(HAB_baseline)
HAB_tunnel.create_ground_level(habitat_axis_height=-3)

HAB_lava_tube = copy.deepcopy(HAB_baseline)
HAB_lava_tube.create_earthworks()

In [ ]:
HAB_baseline.draw(size=(16,16))


In [ ]:
HAB_tunnel.draw(size=(10,10))
HAB_sintered_shield.draw(size=(10,10))

## Configuration Creation

Configurations for both the vertical and horizontal cases are created and executed

In [ ]:
cm_vertical = ConfigurationManager(HAB_vertical, equator, {"T_habitat":list(range(273, 313, 1))})
configs_vertical, heats_vertical, reports_vertical = cm_vertical.run_all_configurations(verbose=True)


In [ ]:
cm_horizontal = ConfigurationManager(HAB_horizontal, equator, {"T_habitat":list(range(273, 313, 1))})
configs_horizontal, heats_horizontal, reports_horizontal = cm_horizontal.run_all_configurations(verbose=True)

## Results

The initial results set is the power loss-temperature plot. These show the range of heat dissipations for a given internal wall temperature, based on an extended habitable range. This gives the surprising result - at noon with average temperatures, an aluminium-skinned habitat will gain rather than lose heat! This is a most unexpected result, as many have an understanding that Mars is a cold environment rather than a hot one. This is true for both horizontal and vertically oriented habitats, although horizontal loses slightly more heat than vertical.

In [ ]:
habitat_temps = [(config["T_habitat"]-273) for config in configs_vertical]

plt.scatter(heats_vertical, habitat_temps, label="Vertical configuration")
plt.scatter(heats_horizontal, habitat_temps, label="Horizontal configuration, base slightly buried")

plt.xlabel("Heat flux (W)")
plt.ylabel("Habitat internal temperature (C)")
plt.title("Internal temperature vs heat loss")

plt.legend()
plt.show()

### Breakdown of heat sources and sinks

The below code accesses the functionality of `syrtis.tools` to plot a "power balance chart" for the two configurations of Habitat. The downward stacked bar below zero shows sources of heat gain into the habitat - primarily direct and indirect solar gain. The upward stacked bar, which begins at some distance below the x-axis, shows the heat sinks present in the system. The most significant of these are radiative losses to the sky and ground, although convection is significant for both and ground conduction plays a reasonable role in the horizontal system. 

This also elucidates the unexpected heat gain result. The large surface area of both habitats leads to a high heat gain from direct sunlight, but also from indirect/reflected sunlight scattered off the Martian surface.

The low emissivity of shiny aluminium leads to the small magnitudes of radiative heat loss. In fact, they are of similar magnitudes to convective and conductive fluxes.

This breakdown makes clear the cause of the difference between the vertical and horizontal configurations. When vertical, the exposed surface area is greater and so indirect solar absorption is greater. 

In [ ]:
vertical_report_293 = [reports_vertical[i] for i in range(len(reports_vertical)) 
if (configs_vertical[i]["T_habitat"] == 293)][0]

horizontal_report_293 = [reports_horizontal[i] for i in range(len(reports_horizontal)) 
if (configs_horizontal[i]["T_habitat"] == 293)][0]


plt.figure(figsize=(8,6))
plt.title("Heat loss breakdown \n Vertical and Horizontal Mars Direct-style Habitats")
tools.plot_power_balance([vertical_report_293, horizontal_report_293], labels=["Vertical", "Horizontal"])

# Modifying the Simulations Post-hoc

In light of this investigation, the user may wish to alter the properties of the geometry or configuration. In particular, it may be desired to apply a surface coating to the outside of the aluminium skin to alter the radiative properties. This can be carried out easily, and then the appropriate simulations re-run.

Consider that a designer wished to maximise heat loss from a habitat, perhaps because it housed a heat-intensive ECLSS or ISRU process. They may choose to apply a similar coating as used on GEO satellites and Mars rovers' MMRTGs to promote heat loss.

In [ ]:
# Coating the aluminum with AZ Technoloy's AZ-2100-ICEW coating
coated_aluminium = copy.deepcopy(aluminium)
coated_aluminium.absorb = 0.28
coated_aluminium.emit = 0.90

HAB_horizontal._shells[-1].material = coated_aluminium
HAB_vertical._shells[-1].material = coated_aluminium

In [ ]:
configs_vertical_coated, heats_vertical_coated, reports_vertical_coated = cm_vertical.run_all_configurations(verbose=True)
configs_horizontal_coated, heats_horizontal_coated, reports_horizontal_coated = cm_horizontal.run_all_configurations(verbose=True)

As could be expected, dramatically increasing the emissivity of the outer layer of the habitat (by a factor of more than 22) causes heat loss to increase hugely. This can be seen in the breakdown below to be almost entirely due to the increase in radiative losses to the sky and ground - although solar gain does also increase slightly. 

In [ ]:
habitat_temps = [(config["T_habitat"]-273) for config in configs_vertical]

plt.scatter(heats_vertical_coated, habitat_temps, label="Vertical configuration")
plt.scatter(heats_horizontal_coated, habitat_temps, label="Horizontal configuration, base slightly buried")

plt.xlabel("Heat flux (W)")
plt.ylabel("Habitat internal temperature (C)")
plt.title("Internal temperature vs heat loss")

plt.legend()
plt.show()

In [ ]:
vertical_coated_report_293 = [reports_vertical_coated[i] for i in range(len(reports_vertical_coated)) 
if (configs_vertical_coated[i]["T_habitat"] == 293)][0]

horizontal_coated_report_293 = [reports_horizontal_coated[i] for i in range(len(reports_horizontal_coated)) 
if (configs_horizontal_coated[i]["T_habitat"] == 293)][0]


plt.figure(figsize=(8,6))
plt.title("Heat loss breakdown \n Vertical and Horizontal Mars Direct-style Habitats")
tools.plot_power_balance([vertical_coated_report_293, horizontal_coated_report_293], labels=["Vertical", "Horizontal"])